# Задания

В самостоятельном проекте вам нужно проанализировать данные о фондах и инвестициях и написать запросы к базе. Задания будут постепенно усложняться, но всё необходимое для их выполнения: операторы, функции, методы работы с базой — вы уже изучили на курсе. К каждому заданию будет небольшая подсказка: она направит вас в нужную сторону, но подробного плана действий не предложит.

1. Посчитайте, сколько компаний закрылось.

In [ ]:
SELECT COUNT(id)
FROM company
WHERE status = 'closed';

2. Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

In [ ]:
SELECT funding_total
FROM company
WHERE category_code = 'news' AND country_code = 'USA'
ORDER BY funding_total DESC;

3. Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE EXTRACT(YEAR FROM (CAST(acquired_at AS date))) BETWEEN 2011 AND 2013
AND term_code = 'cash';

4. Отобразите имя, фамилию и названия аккаунтов людей в поле network_username, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';

5. Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле network_username содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
AND last_name LIKE 'K%';

6. Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code, SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;

7. Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату. Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT CAST(funded_at AS date),
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY CAST(funded_at AS date)
HAVING MIN(raised_amount) <> 0
AND MIN(raised_amount) <>  MAX(raised_amount);

8. Создайте поле с категориями:

* Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
* Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
* Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.

Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,
    CASE
        WHEN invested_companies >= 100 THEN 'high_activity'
        WHEN invested_companies >=20 AND invested_companies < 100 THEN 'middle_activity'
        WHEN invested_companies < 20 THEN 'low_activity'
    END
FROM fund;

9. Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT 
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds));

10. Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 

Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 

Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
    MAX(invested_companies),
    MIN(invested_companies),
    AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY AVG(invested_companies) DESC,
         country_code
LIMIT 10;

11. Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT people.first_name,
       people.last_name,
       education.instituition
FROM people
LEFT JOIN education ON education.person_id=people.id;

12. Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
SELECT company.name,
       COUNT(DISTINCT(instituition))
FROM company
LEFT JOIN people ON company.id=people.company_id
LEFT JOIN education ON people.id=education.person_id
GROUP BY company.id
ORDER BY COUNT(DISTINCT(instituition)) DESC
LIMIT 5;

13. Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT(company.name)
FROM funding_round
LEFT JOIN company ON company.id=funding_round.company_id
WHERE is_first_round = 1 AND is_last_round = 1
AND company.status LIKE '%closed%';

14. Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT DISTINCT(people.id)
FROM people 
WHERE people.company_id IN
(SELECT DISTINCT(company.id)
FROM company
JOIN funding_round ON company.id=funding_round.company_id
WHERE is_first_round = 1 AND is_last_round = 1
AND company.status LIKE '%closed%'
GROUP BY company.id);

15. Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT people.id,
       education.instituition
FROM people
LEFT JOIN education ON education.person_id=people.id
WHERE people.company_id IN(
SELECT DISTINCT(company.id)
FROM company
JOIN funding_round ON company.id=funding_round.company_id
WHERE is_first_round = 1 AND is_last_round = 1
AND company.status LIKE '%closed%'
GROUP BY company.id)
GROUP BY people.id,
          education.instituition
HAVING instituition IS NOT NULL;

16. Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT people.id,
       COUNT(education.instituition)
FROM people
LEFT JOIN education ON education.person_id=people.id
WHERE people.company_id IN(
SELECT DISTINCT(company.id)
FROM company
JOIN funding_round ON company.id=funding_round.company_id
WHERE is_first_round = 1 AND is_last_round = 1
AND company.status LIKE '%closed%'
GROUP BY company.id)
GROUP BY people.id
HAVING COUNT(DISTINCT education.instituition) > 0;

17. Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
WITH 
cose AS 
(SELECT people.id,
       COUNT(education.instituition)
FROM people
LEFT JOIN education ON education.person_id=people.id
WHERE people.company_id IN(
SELECT DISTINCT(company.id)
FROM company
JOIN funding_round ON company.id=funding_round.company_id
WHERE is_first_round = 1 AND is_last_round = 1
AND company.status LIKE '%closed%'
GROUP BY company.id)
GROUP BY people.id
HAVING COUNT(DISTINCT education.instituition) > 0)
SELECT AVG(COUNT)
FROM cose;

18. Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.
*(сервис, запрещённый на территории РФ)

In [ ]:
WITH 
cose AS 
(SELECT people.id,
       COUNT(education.instituition)
FROM people
LEFT JOIN education ON education.person_id=people.id
WHERE people.company_id IN(
SELECT DISTINCT(company.id)
FROM company
WHERE name = 'Facebook'
GROUP BY company.id)
GROUP BY people.id
HAVING COUNT(DISTINCT education.instituition) > 0)
SELECT AVG(COUNT)
FROM cose
;

19. Составьте таблицу из полей:

* name_of_fund — название фонда;
* name_of_company — название компании;
* amount — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT fund.name AS name_of_fund,
       company.name AS name_of_company,
       fund_round.raised_amount AS amount
FROM investment
LEFT JOIN company ON company.id=investment.company_id
LEFT JOIN fund ON fund.id=investment.fund_id
JOIN
(SELECT *
FROM funding_round
WHERE EXTRACT(YEAR FROM CAST(funded_at AS date)) BETWEEN 2012 AND 2013)
AS fund_round ON fund_round.id=investment.funding_round_id
WHERE company.milestones > 6;

20. Выгрузите таблицу, в которой будут такие поля:

* название компании-покупателя;
* сумма сделки;
* название компании, которую купили;
* сумма инвестиций, вложенных в купленную компанию;
* доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 

Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
WITH
acquiring AS
(SELECT company.name AS buyer,
        acquisition.price_amount AS price,
        acquisition.id AS key
FROM acquisition
LEFT JOIN company ON acquisition.acquiring_company_id=company.id
WHERE acquisition.price_amount > 0),
acquired AS
(SELECT company.name AS acquisition,
        company.funding_total AS investment,
        acquisition.id AS key
FROM acquisition
LEFT JOIN company ON acquisition.acquired_company_id = company.id
WHERE company.funding_total > 0)
SELECT acquiring.buyer,
       acquiring.price,
       acquired.acquisition,
       acquired.investment,
       ROUND(price/investment) AS uplift
FROM acquiring
JOIN acquired ON acquiring.key=acquired.key
ORDER BY price DESC,
        acquisition
LIMIT 10;

21. Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT company.name,
       EXTRACT(MONTH FROM CAST(funding_round.funded_at AS date))   
FROM company
JOIN funding_round ON company.id=funding_round.company_id
WHERE category_code = 'social'
AND EXTRACT(YEAR FROM CAST(funding_round.funded_at AS date)) BETWEEN 2010 AND 2013
AND raised_amount > 0;

22. Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:

* номер месяца, в котором проходили раунды;
* количество уникальных названий фондов из США, которые инвестировали в этом месяце;
* количество компаний, купленных за этот месяц;
* общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH
fundd AS 
(SELECT EXTRACT(MONTH FROM CAST(funding_round.funded_at AS date)) AS funding_month,
       COUNT(DISTINCT fund.id) AS unic_name
FROM fund
LEFT JOIN investment ON fund.id=investment.fund_id
LEFT JOIN funding_round ON investment.funding_round_id= funding_round.id
WHERE fund.country_code = 'USA'
AND EXTRACT(YEAR FROM CAST(funding_round.funded_at AS date)) BETWEEN 2010 AND 2013
GROUP BY funding_month),
acquisitions AS
(SELECT EXTRACT(MONTH FROM CAST(acquired_at AS date)) AS funding_month,
        COUNT(acquired_company_id) AS count_comp,
        SUM(price_amount) AS sum_sdel
FROM acquisition
WHERE EXTRACT(YEAR FROM CAST(acquired_at AS date)) BETWEEN 2010 AND 2013
GROUP BY funding_month)
SELECT fundd.funding_month, fundd.unic_name, acquisitions.count_comp, acquisitions.sum_sdel
FROM fundd
LEFT JOIN acquisitions ON fundd.funding_month=acquisitions.funding_month;

23. Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
y2011 AS 
(SELECT country_code AS country,
        AVG(funding_total) AS avg2011
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2011
GROUP BY country_code),
y2012 AS 
(SELECT country_code AS country, 
        AVG(funding_total) AS avg2012
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2012
GROUP BY country_code),
y2013 AS (SELECT country_code AS country, 
          AVG(funding_total) AS avg2013
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2013
GROUP BY country_code)

SELECT y2011.country, 
       y2011.avg2011, 
       y2012.avg2012, 
       y2013.avg2013
FROM y2011
INNER JOIN y2012 ON y2011.country = y2012.country
INNER JOIN y2013 ON y2011.country = y2013.country
ORDER BY y2011.avg2011 DESC;